In [112]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
import os


In [113]:
plt.rc('font', family='serif', size=14)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=14)
output_path = "D:/Dissetation/overleaf/dissertation/pics"
# output_path = "./output_figures"
os.makedirs(output_path, exist_ok=True)
from matplotlib.gridspec import GridSpec

In [114]:
def forceParamsPlot(data, y, hue, output_path):
    fig = plt.figure(figsize=(15, 5))
    gs = GridSpec(1, 3, width_ratios=[8, 1, 1])     
    # Create subplots using GridSpec
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    ax3 = fig.add_subplot(gs[2])
    sns.barplot(ax=ax1, data=data, x='Muscle', hue=hue, y=y, errorbar='ci', capsize=.4,
    err_kws={"color": "0.2", "linewidth": 1},)
    ax1.legend()
    if y=='Gain':
        ax1.set_yticks([0, 1], labels=['0.01', '10'])
    elif y=='GainRatio':
        ax1.set_yticks([0, 1], labels=['6', '131'])
    
    ax1.set_ylabel(y,labelpad=-20)

    ax1.hlines(y=0, xmin=-1, xmax=10, color='black', linestyles='dotted')
    ax1.hlines(y=1, xmin=-1, xmax=10, color='black', linestyles='dotted')
    ax1.set_xlim(-0.5, 9.5)
    plt.setp(ax1.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    ax1.legend(loc='upper left', bbox_to_anchor=(0.1, 1.2), ncol=2)
    ax1.set_ylabel(y,labelpad=-20)
    sns.barplot(ax=ax2, data=data, hue=hue, y='forceVelocity', errorbar='ci', capsize=.4,
        err_kws={"color": "0.2", "linewidth": 1},)
    ax2.set_yticks([0.15, 0.25], labels=['0.15', '0.25'])
    ax2.hlines(y=0.15, xmin=-1, xmax=0.5, color='black', linestyles='dotted')
    ax2.hlines(y=0.25, xmin=-1, xmax=0.5, color='black', linestyles='dotted')
    ax2.set_xlim(-0.5, 0.5)
    ax2.set_ylim(0.15, 0.255)
    ax2.set_ylabel('Force Velocity',labelpad=-20)
    ax2.legend().set_visible(False)
    sns.barplot(ax=ax3, data=data, hue=hue, y='passiveCoefficient', errorbar='ci', capsize=.4,
        err_kws={"color": "0.2", "linewidth": 1},)
    ax3.set_yticks([0.05, 0.25], labels=['0.05', '0.25'])
    ax3.hlines(y=0.05, xmin=-1, xmax=0.5, color='black', linestyles='dotted')
    ax3.hlines(y=0.25, xmin=-1, xmax=0.5, color='black', linestyles='dotted')
    ax3.set_xlim(-0.5, 0.5)
    ax3.set_ylim(0.05, 0.26)
    ax3.set_ylabel('Passive Coefficient',labelpad=-20)
    ax3.legend().set_visible(False)
    plt.show()
    filepath = os.path.join(output_path, 'Forceparameters')
    # plt.savefig(filepath, bbox_inches='tight', dpi=300)
    plt.close()


In [115]:

# Load data
path = "E:/Quanitifying EMG/Summary-Force Parameters.csv"
data = pd.read_csv(path)
sorted_decades = sorted(data['Decade'].unique())
data['Status'] = data['Status'].replace('Control', 'Asymptomatic')
data['Decade'] = pd.Categorical(data['Decade'], categories=sorted_decades, ordered=True)
data['Muscle'] = data['Muscle'].replace('RightErectorSpinae', 'RES')
data['Muscle'] = data['Muscle'].replace('LeftErectorSpinae', 'LES')
data['Muscle'] = data['Muscle'].replace('RightInternalOblique', 'RIO')
data['Muscle'] = data['Muscle'].replace('LeftInternalOblique', 'LIO')
data['Muscle'] = data['Muscle'].replace('RightLatissimusDorsi', 'RLD')
data['Muscle'] = data['Muscle'].replace('LeftLatissimusDorsi', 'LLD')
data['Muscle'] = data['Muscle'].replace('RightExternalOblique', 'REO')
data['Muscle'] = data['Muscle'].replace('LeftExternalOblique', 'LEO')
data['Muscle'] = data['Muscle'].replace('RightRectusAbdominis', 'RRA')
data['Muscle'] = data['Muscle'].replace('LeftRectusAbdominis', 'LRA')
data['GainRatio'] = (data['GainRatio'] - 6) / (131- 6)
data['Gain'] = (data['Gain'] - 0.01) / (10 - 0.01)
# Define options for dropdowns
y_options = ['Gain', 'GainRatio']
hue_options = ['Status','Sex',]

# Create dropdown widgets
y_dropdown = widgets.Dropdown(options=y_options, value='Gain', description='Y-axis:')
hue_dropdown = widgets.Dropdown(options=hue_options, value='Status', description='Hue:')

# Define filter widgets
filter_column_options = data.columns.tolist()  # All columns in the data
default_filter_column1 = 'Subject'  # Default column to filter by
default_filter_values1 = data[default_filter_column1].unique().tolist()  # Default filter values

# Create filter widgets
filter_column_dropdown1 = widgets.Dropdown(options=filter_column_options, value=default_filter_column1, description='Filter by:')
filter_values_select1 = widgets.SelectMultiple(options=default_filter_values1, description='Filter Values:', disabled=False)

# Function to update filter values based on the selected column
def update_filter_values1(change):
    """
    Update the filter values when the filter column is changed.
    """
    if change['new'] and change['new'] in data.columns:
        unique_values = data[change['new']].unique()
        sorted_values = sorted(unique_values, key=lambda x: str(x))
        filter_values_select1.options = sorted_values
        filter_values_select1.disabled = False
    else:
        filter_values_select1.options = []
        filter_values_select1.disabled = True

# Attach the update function to the filter column dropdown
filter_column_dropdown1.observe(update_filter_values1, names='value')

# Modify the update_plot function to include filtering
def update_plot( y, hue, filter_column, filter_values):
    """
    Update the plot based on the selected filters and plot parameters.
    """
    # Apply filters
    if filter_column and filter_values:
        filtered_data = data[data[filter_column].isin(filter_values)]
    else:
        filtered_data = data
    
    # Generate the plot
    forceParamsPlot(filtered_data, y, hue, output_path)

# Add filter widgets to the controls group
controls_group1 = widgets.HBox([
    widgets.VBox([
        widgets.HTML('<b>Plot Controls</b>'),
        y_dropdown,
        hue_dropdown,
    ]),
    widgets.VBox([
        widgets.HTML('<b>Filter Controls</b>'),
        filter_column_dropdown1,
        filter_values_select1,
    ])
], layout=Layout(margin='10px'))

# Create widgets container
widgets_container = widgets.HBox(
    [controls_group1,],
    layout=Layout(
        display='inline-flex',
        flex_flow='row nowrap',
        align_items='flex-start',
        justify_content='space-around',
        width='100%'
    )
)

# Display widgets container
display(widgets_container)

# Connect dropdowns to the update_plot function
out = widgets.interactive_output(
    update_plot,
    {
        'y': y_dropdown,
        'hue': hue_dropdown,
        'filter_column': filter_column_dropdown1,
        'filter_values': filter_values_select1,
    }
)

# Display the output
display(out)

Output()